# Load data

In [13]:
import pandas as pd

df = pd.read_csv('data/train_time.csv')
df = df[:100]
df.head()

,id,sentence,date,birthday
0,100022_0,Болеет СД 2 типа в течении 5 лет.,2010-12-14 18:33:23,1957-01-01
1,100085_0,"Считает себя больной с 1993 года, когда перене...",2010-12-14 21:06:48,1950-11-05
2,100085_4,В мае 2009 года в СибГМУ проводилась РЧ-аблаци...,2010-12-14 21:06:48,1950-11-05
3,100085_5,"Ухудшение в конце 2009, начале 2010 года, когд...",2010-12-14 21:06:48,1950-11-05
4,100085_7,В апреле 2010 года проходила лечение в отд НРС...,2010-12-14 21:06:48,1950-11-05


# Parse single sentence

In [2]:
from syntax.parser import Parser

parser = Parser()


[nltk_data] Downloading package punkt to /home/renoip2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/renoip2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/renoip2/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/renoip2/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
Initialization complete!


In [3]:
from TimeExpressions.TimeProcessor import TimeProcessor

processor = TimeProcessor()

In [4]:
doc = processor.process(sentence="Болеет СД 2 типа в течении 5 лет .", date='2010-05-17 00:00:00', birthday='1944-04-19', parser=parser)
print([('ВК: '+ent.text, 'Метка: '+str(ent._.timestamp), 'Нормальная форма: '+str(ent._.normal_form), 'Событие: '+ent._.event) for ent in doc[0].ents])

[('ВК: в течении 5 лет', 'Метка: 2', 'Нормальная форма: [datetime.datetime(2005, 5, 17, 0, 0), datetime.datetime(2010, 5, 17, 0, 0)]', 'Событие: Болеет СД 2 типа')]


# Parse set of sentences and convert result to pandas

In [15]:
result = processor.process(sentence=df.sentence.values, date=df.date.values, birthday=df.birthday.values, to_dataframe=True, parser=parser)
result.head()

,sentence,date,birthdate,time_expr,event,norm,uncertain,stamp,rule
0,Болеет СД 2 типа в течении 5 лет .,2010-12-14 18:33:00,1957-01-01 00:00:00,в течении 5 лет,Болеет СД 2 типа,2005-12-14 18:33:00 – 2010-12-14 18:33:00,"[2004-12-14 18:33:00, 2005-12-14 18:33:00, 201...",[2],r_dur_int_unit
1,"Считает себя больной с 1993 года , когда перен...",2010-12-14 21:06:00,1950-11-05 00:00:00,с 1993 года,перенесла гинекологическую операцию,1993-07-01 00:00:00 – 2010-12-14 21:06:00,"[1992-07-01 00:00:00, 1993-07-01 00:00:00, 201...",[2],r_from_year4d_year
2,В мае 2009 года в СибГМУ проводилась РЧ - абла...,2010-12-14 21:06:00,1950-11-05 00:00:00,В мае 2009 года,проводилась РЧ - аблация по поводу ЖЭ,2009-05-15 00:00:00,"[2009-04-15 00:00:00, 2009-05-15 00:00:00, 200...",[1],r_in_month_yeard4d_year
3,"Ухудшение в конце 2009 , начале 2010 года , ко...",2010-12-14 21:06:00,1950-11-05 00:00:00,"в конце 2009 , начале 2010",Ухудшение,2010-02-15 00:00:00,"[2010-01-01 00:00:00, 2010-02-15 00:00:00, 201...",[1],r_in_part_year4d_part_year4d
4,В апреле 2010 года проходила лечение в отд НРС...,2010-12-14 21:06:00,1950-11-05 00:00:00,"В апреле 2010 года, за сутки","проходила лечение в отд НРС, холтере 1 экстрас...","2010-04-15 00:00:00, 2010-12-13 21:06:00 – 201...","[2010-03-15 00:00:00, 2010-04-15 00:00:00, 201...","[1, 2]","r_in_month_yeard4d_year, r_za_unit"


# Example of negation module

In [16]:
import pandas as pd

df = pd.read_csv('data/train_neg.csv', index_col=0)
df = df[:100]
df.head()

,sentences,form,entity
0,Со слов больного ранее боли в области сердца н...,"боли не отмечал, АД не контролировал","боли, АД"
1,"На ЭКГ элевация ST V1-V3 , с представлением об...",не принимает медикаменты,медикаменты
2,За время наблюдения самочувствие без ухудшения...,"без ухудшения самочувствие, без особенностей м...",ухудшения
3,"Не обследовалась , антиангинальной терапии не ...","Не обследовалась, не проводилось антиангинальн...",обследовалась
4,На контрольной КГ- отмечается некоторое усугуб...,не выполнять дилатацию устья,дилатацию устья


In [2]:
from syntax.parser import Parser

parser = Parser()


[nltk_data] Downloading package punkt to /home/renoip2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/renoip2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/renoip2/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/renoip2/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
Initialization complete!


In [17]:
from negations import negations

negator = negations.Negator()

In [18]:
result = negator.process(sentence=df.sentences.values, parser=parser, to_dataframe=True)
result.head()

,sentence,neg_expr,neg_ent
0,Со слов больного ранее боли в области сердца н...,"не отмечал, не контролировал АД",", АД"
1,"На ЭКГ элевация ST V1-V3 , с представлением об...",не принимает медикаменты,медикаменты
2,За время наблюдения самочувствие без ухудшения...,"без ухудшения самочувствие, без особенностей м...","ухудшения самочувствие, особенностей место пун..."
3,"Не обследовалась , антиангинальной терапии не ...","Не обследовалась, не проводилось антиангинальн...",", антиангинальной терапии"
4,На контрольной КГ - отмечается некоторое усугу...,не выполнять дилатацию устья,дилатацию устья
